**Question 4**

---

Using the 'IBRD_Balance_Sheet__FY2010.csv', create a table and populate it with its contents, specify a data store which will host the cube’s data and modify the "tutorial_model.json" file to include aggregate measures for the minimum and maximum amount in the data cube. Using these implemented aggregate measures, produce the values for the minimum and maximum amount in the data per year.

**Solution**

---

I modified the ‘tutorial_model.json’ as below to include aggregate measures for the minimum and maximum amount and produce the values for the minimum and maximum amount in the data per year.

In [1]:
# If you are using Google Colab, you would need to run the below line to install Cubes. 
# You can comment the below line if you are running a local python installation with Cubes installed.
!pip install cubes
#!pip uninstall sqlalchemy

!pip install sqlalchemy==0.7.4

from sqlalchemy import create_engine
from cubes.tutorial.sql import create_table_from_csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 128 kB 5.6 MB/s 
     |████████████████████████████████| 82 kB 252 kB/s 
  Created wheel for cubes: filename=cubes-1.1-py3-none-any.whl size=151588 sha256=bd66413e50110dbd88e4a113f312605af1c42392f490fca3c6b9ec84eac6b6d9
  Stored in directory: /root/.cache/pip/wheels/db/b6/92/d26c73b4e7c78d7d6a867063470b8f1ea024c3b6064a063080
  Created wheel for expressions: filename=expressions-0.2.3-py3-none-any.whl size=6444 sha256=7b05a0c73da52883a63b4325d01586ce1daaf541d35b5e5fbb61ae2887297079
  Stored in directory: /root/.cache/pip/wheels/cd/76/e0/cb8c2b868349a76bc6458821bbb9d08f484b097038f78879e6
Successfully built cubes expressions
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.5 MB 5.3 MB/s 
  Created wheel for sqlalchemy: filename=SQLAlchemy-0.7.4-py3-none-any.whl 

In [3]:
#We can now load the data, create a table and populate it with contents of the CSV file.
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "IBRD_Balance_Sheet__FY2010.csv",
                      table_name="ibrd_balance",
                      fields=[
                          ("category", "string"),
                          ("category_label", "string"),
                          ("subcategory", "string"),
                          ("subcategory_label", "string"),
                          ("line_item", "string"),
                          ("year", "integer"),
                          ("amount", "integer")],
                      create_id=True
                     )

In [4]:
#Specify a data store – a database which will host the cube’s data
from cubes import Workspace

workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

In [5]:
#Import file 'tutorial_model.json', which includes an example model of the data cube, 
#dimension tables, and aggregate functions for the CSV file we loaded previously.
workspace.import_model("tutorial_model.json")

In [6]:
#Create a data cube based on the above data cube model and data table
cube = workspace.cube("ibrd_balance")

In [7]:
#We use browser object to do the actual aggregations and other data queries for a cube.
browser = workspace.browser(cube)

In [8]:
#drilldown operation to year dimension
result = browser.aggregate(drilldown=["year"]) 
for record in result:
  print(record)

{'year': 2009, 'amount_min': -1683, 'amount_max': 110040, 'amount_sum': 550840, 'amount_avg': 17769.032258064515, 'record_count': 31}
{'year': 2010, 'amount_min': -3043, 'amount_max': 128577, 'amount_sum': 566020, 'amount_avg': 18258.709677419356, 'record_count': 31}


**Question 2**

---

Using the CSV file "country-income.csv" (found in the supplementary lab documents), perform the following:
  1. Load the CSV file using Cubes, create a JSON file for the data cube model, and create a data cube for the data. Use as dimensions the region, age, and online shopper fields. Use as measure the income. Define aggregate functions in the data cube model for the total, average, minimum, and maximum income. In your PDF report, show the relevant scripts and files created.
  2. Using the created data cube and data cube model, produce aggregate results for: the whole data cube; results per region; results per online shopping activity; and results for all people aged between 40 and 50.

**Solution**

---


In [9]:
#We can now load the data, create a table and populate it with contents of the CSV file.
engine = create_engine('sqlite:///data.sqlite')
create_table_from_csv(engine,
                      "country-income.csv",
                      table_name="countryIncome",
                      fields=[
                          ("region", "string"),
                          ("age", "integer"),
                          ("income", "integer"),
                          ("shopper", "string")],
                      create_id=True
                     )

In [10]:
#Specify a data store – a database which will host the cube’s data
workspace = Workspace()
workspace.register_default_store("sql", url="sqlite:///data.sqlite")

In [11]:
#Import file 'tutorial_model.json', which includes an example model of the data cube, 
#dimension tables, and aggregate functions for the CSV file we loaded previously.
workspace.import_model("countryModel.json")

In [12]:
#Create a data cube based on the above data cube model and data table
cube = workspace.cube("countryIncome")

In [13]:
#We use browser object to do the actual aggregations and other data queries for a cube.
browser = workspace.browser(cube)

In [14]:
result = browser.aggregate()

#Display total income.
result.summary["income_sum"]

#Display average income.
result.summary["income_avg"]

#Display maximum income.
result.summary["income_max"]

#Display minimum income.
result.summary["income_min"]

57600

2. *Aggregate results for all people aged between 40-50*

In [16]:
import cubes as cubes

#RangeCut function to select the ages between 40 and 50 
cuts = [cubes.RangeCut("age", from_path=[40], to_path=[50])] 
cell = cubes.Cell(cube, cuts)

#drilldown operation to age dimension
result = browser.aggregate(cell, drilldown=["age"])
for record in result:
  print(record)

{'age': 40, 'income_min': 69600, 'income_max': 69600, 'income_sum': 69600, 'income_avg': 69600.0, 'record_count': 1}
{'age': 42, 'income_min': 80400, 'income_max': 80400, 'income_sum': 80400, 'income_avg': 80400.0, 'record_count': 1}
{'age': 43, 'income_min': 73200, 'income_max': 73200, 'income_sum': 73200, 'income_avg': 73200.0, 'record_count': 1}
{'age': 45, 'income_min': 79400, 'income_max': 79400, 'income_sum': 79400, 'income_avg': 79400.0, 'record_count': 1}
{'age': 46, 'income_min': 62400, 'income_max': 62400, 'income_sum': 62400, 'income_avg': 62400.0, 'record_count': 1}
{'age': 49, 'income_min': 86400, 'income_max': 86400, 'income_sum': 86400, 'income_avg': 86400.0, 'record_count': 1}


In [17]:
#drilldown operation to online shopping activity dimension
result = browser.aggregate(drilldown=["shopper"])
for record in result:
  print(record)

{'shopper': 'No', 'income_min': 62400, 'income_max': 99600, 'income_sum': 386400, 'income_avg': 77280.0, 'record_count': 5}
{'shopper': 'Yes', 'income_min': 57600, 'income_max': 94800, 'income_sum': 381800, 'income_avg': 76360.0, 'record_count': 5}


In [18]:
#drilldown operation to region dimension
result = browser.aggregate(drilldown=["region"]) 
for record in result:
  print(record)

{'region': 'Brazil', 'income_min': 57600, 'income_max': 73200, 'income_sum': 193200, 'income_avg': 64400.0, 'record_count': 3}
{'region': 'India', 'income_min': 69600, 'income_max': 94800, 'income_sum': 331200, 'income_avg': 82800.0, 'record_count': 4}
{'region': 'USA', 'income_min': 64800, 'income_max': 99600, 'income_sum': 243800, 'income_avg': 81266.66666666667, 'record_count': 3}
